# **CHATBOT**

Este código nos permite crear un chatbot inteligente que ayuda a los usuarios a conocer servicios de software por sectores. El chatbot puede entender lo que preguntamos, mostrar servicios disponibles, y registrar personas interesadas. Es un asistente virtual que trabaja con datos de un Excel y entiende español básico.



1. **Configuración**: Instala herramientas y carga los datos del Excel
2. **Organización**: Ordena los servicios por sector para encontrarlos fácilmente
3. **Entendimiento**: Define palabras clave para saber lo que el usuario quiere
4. **Memoria**: Carga el historial de clientes anteriores
5. **Inteligencia**: La función que analiza y entiende los mensajes
6. **Respuesta**: Funciones que muestran servicios y registran interesados
7. **Conversación**: El chatbot principal que maneja el diálogo



In [1]:
# =============================================
# CONFIGURACIÓN INICIAL Y CARGA DE DATOS
# =============================================

# Instalar y cargar las herramientas necesarias para entender lenguaje natural

!pip install -q spacy

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 27.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# **Proyecto**

In [5]:
import pandas as pd
import os
import spacy

# Cargar el modelo en español que entiende palabras y su significado
nlp = spacy.load("es_core_news_sm")

# Cargar el archivo Excel con todos los servicios disponibles
df_servicios = pd.read_excel("servicios.xlsx")

# =============================================
# ORGANIZAR LOS SERVICIOS POR SECTOR
# =============================================
# Crear un diccionario vacío para guardar los servicios organizados por sector
#Se recorre cada fila del DataFrame y se agrupa la información en un diccionario servicios_por_sector,
#con el nombre del sector como clave y una lista de diccionarios con los servicios como valor.
servicios_por_sector = {}
# Recorrer cada fila del Excel y organizar la información
for _, row in df_servicios.iterrows():
    sector = row['Sector'].strip().lower() # Convertir el nombre del sector a minúsculas
    if sector not in servicios_por_sector:
        servicios_por_sector[sector] = [] # Crear lista vacía si el sector es nuevo
  # Agregar la información del servicio a su sector correspondiente
    servicios_por_sector[sector].append({
        'ID': row['ID'],
        'Nombre': row['Nombre del servicio'],
        'Descripción': row['Descripción'],
        'Tecnología usada': row['Tecnología usada'],
        'Tipo de solución': row['Tipo de solución'],
        'Palabras clave': row['Palabras clave'],
        'Disponible': row['Disponible']
    })

# =============================================
# PALABRAS CLAVE PARA ENTENDER AL USUARIO
# =============================================

# Diccionario con palabras relacionadas a cada sector
# Esto ayuda al chatbot a saber de qué sector estás hablando
palabras_clave_por_sector = {
    "turismo": ["viaje", "turismo", "hotel", "destino", "vacaciones"],
    "agroindustria": ["campo", "agroindustria" "agricultura", "agro", "cultivo", "tierra", "maquinaria"],
    "energía": ["energía", "energia","electricidad", "red", "equipo", "infraestructura"],
    "financiero": ["banco", "financiero", "dinero", "finanzas", "transacción", "activos"],
    "salud": ["salud", "clínica", "hospital", "diagnóstico", "historia", "paciente"],
    "educación": ["educación", "educacion","escuela", "aula", "aprendizaje", "clase", "estudio"]
}

#palabras clave para determinar la intención
# Listas de palabras para detectar lo que el usuario quiere (intenciones)
ubicacion_keywords = ["ubicación", "ubicacion", "dónde", "donde", "encuentran", "ubicado", "lugar", "localización"]
horario_keywords = ["horario", "hora", "cuándo", "tiempo", "atienden"]
disponibilidad_keywords = ["disponible", "disponibilidad", "ofrecen", "activo"]
servicios_keywords = ["servicio", "servicios", "ofrece", "ofrecen", "qué", "producto", "productos", "solucion", "soluciones"]
interes_keywords = ["interesado", "quiero", "me interesa", "cotizar", "comprar", "necesito", "agenda", "entrevista"]
saludo_keywords = ["hola", "buenos", "días", "buenas", "tardes", "noches", "saludos", "hey"]
despedida_keywords = ["adiós", "adios", "chao", "hasta luego", "nos vemos", "hasta pronto", "salir"]
agradecimiento_keywords = ["gracias", "muchas gracias", "te lo agradezco", "eres amable"]


# =============================================
# HISTORIAL DE CLIENTES INTERESADOS
# =============================================

# Archivo donde se guardarán las personas interesadas
# Historial de clientes interesados
archivo_historico = "clientes_interesados.xlsx"

# Cargar el historial si existe, o crear una lista vacía si es la primera vez
if os.path.exists(archivo_historico):
    clientes_interesados = pd.read_excel(archivo_historico).to_dict('records')
else:
    clientes_interesados = []


# =============================================
# FUNCIÓN PARA ENTENDER LO QUE QUIERE EL USUARIO
# =============================================
# Función para detectar intención
def detectar_intencion(texto):
#Esta función analiza el texto del usuario y decide qué es lo que quiere
# Convertir el texto a minúsculas y separarlo en palabras importantes
    doc = nlp(texto.lower())
    tokens = [token.lemma_ for token in doc] # Lema = forma básica de la palabra

# Verificar cada tipo de intención comparando con las palabras clave
    if any(p in tokens for p in servicios_keywords):
        return "servicios"
    if any(p in tokens for p in saludo_keywords):
        return "saludo"
    if any(p in tokens for p in despedida_keywords):
        return "despedida"
    if any(p in tokens for p in agradecimiento_keywords):
        return "agradecimiento"
    if any(p in tokens for p in ubicacion_keywords):
        return "ubicacion"
    if any(p in tokens for p in horario_keywords):
        return "horario"
    if any(p in tokens for p in disponibilidad_keywords):
        return "disponibilidad"
    if any(p in tokens for p in interes_keywords):
        return "interes"

# Verificar si habla de algún sector específico
    for sector, palabras in palabras_clave_por_sector.items():
        if any(p in tokens for p in palabras + [sector]):
            return f"sector_{sector}"
# Si no entendió nada, devolver "desconocido"
    return "desconocido"


# =============================================
# FUNCIONES PARA MOSTRAR SERVICIOS
# =============================================
# Mostrar servicios por sector
def mostrar_servicios_sector(sector_detectado, mostrar_completo=True):
#Muestra los servicios de un sector específico
    if sector_detectado in servicios_por_sector:
        print(f"\n📌 Servicios del sector: {sector_detectado.capitalize()}\n")
        for servicio in servicios_por_sector[sector_detectado]:
            print(f"  Nombre: {servicio['Nombre']}")
            if mostrar_completo:
              # Mostrar información detallada del servicio
                print(f"  Descripción: {servicio['Descripción']}")
                print(f"  Tecnología usada: {servicio['Tecnología usada']}")
                print(f"  Tipo de solución: {servicio['Tipo de solución']}")
                print(f"  Disponible: {servicio['Disponible']}")
            print("-" * 50) # Línea separadora
        print("chatbot 🤖: ¿Estás interesado en algún servicio? Escribe 'estoy interesado'.")
    else:
        print("No hay servicios para ese sector.")

# Mostrar todos los servicios
def mostrar_todos_los_servicios():
  #Muestra todos los servicios disponibles
    print("\n📌 Servicios disponibles:\n")
    for sector, servicios in servicios_por_sector.items():
        print(f"**{sector.capitalize()}**\n")
        for servicio in servicios:
            print(f"  {servicio['Nombre']}") # Solo mostrar el nombre del servicio
        print("-" * 20) # Línea separadora entre sectores
    print("chatbot 🤖: ¿Te interesa algún sector en particular? Escríbelo para mayor información.")

# =============================================
# FUNCIÓN PARA REGISTRAR PERSONAS INTERESADAS
# =============================================
# Función para registrar interesado
def registrar_interesado():
  #Registra a una persona interesada pidiendo sus datos de contacto
    print("Chatbot 🤖: ¡Qué bien! Para agendar una entrevista, necesito algunos datos.")

  # Pedir información básica del cliente
    nombre = input("Chatbot 🤖: ¿Cuál es tu nombre completo? ")
    correo = input("Chatbot 🤖: ¿Cuál es tu correo electrónico? ")
    telefono = input("Chatbot 🤖: ¿Cuál es tu número de teléfono? ")
    interes = input("Chatbot 🤖: ¿Qué servicio o solución te interesa? ")

# Explicar el proceso de trabajo
    print("\n✅ ¡Gracias por compartir tu información! Así continúa nuestro proceso:")
    print("1. 🧠 Ideación: Realizamos la entrevista y recopilamos la información.")
    print("2. ✍️ Propuesta: Te entregamos una arquitectura o review de branding con la solución.")
    print("3. 🤝 Vínculo: Si estás de acuerdo, firmamos contrato.")
    print("4. 👨‍💻 Desarrollo: Nuestro equipo implementa la solución con entregas ágiles.")
    print("5. ✅ Calidad y entrega: Hacemos pruebas y ponemos tu solución en producción.")
    print("6. 💙 Fidelización: Mantenemos el vínculo con mejoras y nuevos desarrollos.")

    print("\n🗓️ Pronto te contactaremos para confirmar fecha y hora de la entrevista. ¡Gracias por confiar en nosotros!")

 # Guardar la información del cliente interesado
    nuevo_interesado = {
        'Nombre': nombre,
        'Correo': correo,
        'Teléfono': telefono,
        'Interés': interes
    }
    clientes_interesados.append(nuevo_interesado)

    # Guardar la información en el archivo Excel
    df_actualizado = pd.DataFrame(clientes_interesados)
    df_actualizado.to_excel(archivo_historico, index=False)


# =============================================
# CHATBOT PRINCIPAL - CEREBRO DEL ASISTENTE
# =============================================

def chatbot():
  #Función principal que controla toda la conversación con el usuario
    print("Chatbot 🤖: ¡Hola! Bienvenido a Soft Art, Soy tu asistente virtual.\n💻Si estás buscando contratar un servicio de software, estás en el lugar correcto.")

    # Bucle infinito que mantiene la conversación activa
    while True:
        entrada = input("Tú: ").strip()# Leer lo que escribe el usuario
        intencion = detectar_intencion(entrada) # Detectar qué quiere el usuario

# Responder según lo que detectó el chatbot
        if intencion == "despedida":
            print("Chatbot 🤖: ¡Gracias por visitarnos! 👋")
            break # Terminar la conversación
        elif intencion == "saludo":
            print("Chatbot 🤖: ¡Hola! Bienvenido a Soft Art, Soy tu asistente virtual. ¿En qué puedo ayudarte hoy?")
        elif intencion == "agradecimiento":
            print("Chatbot 🤖: ¡Con gusto! 😊")
        elif intencion == "ubicacion":
            print("Chatbot 🤖: Estamos ubicados en Calle 7 - Cra. 36 #22a Sur 40, Villavicencio, Meta.")
        elif intencion == "horario":
            print("Chatbot 🤖: Atendemos de lunes a viernes de 8:00am a 6:00pm.")
        elif intencion == "disponibilidad":
            print("Chatbot 🤖: Sí, contamos con servicios activos y disponibles. ¿Te interesa alguno en particular?")
        elif intencion == "servicios":
            mostrar_todos_los_servicios() # Mostrar todos los servicios
        elif intencion.startswith("sector_"):
            sector = intencion.replace("sector_", "") # Extraer el nombre del sector
            mostrar_servicios_sector(sector, mostrar_completo=True) # Mostrar servicios del sector
        elif intencion == "interes":
            registrar_interesado() # Registrar cliente interesado
            break # Terminar después de registrar
        else:
            print("Chatbot 🤖: Lo siento, no entendí tu mensaje. Puedes preguntarme por los servicios, ubicación o decirme si estás interesado.")

# =============================================
# INICIAR EL CHATBOT
# =============================================

# Esta línea hace que el chatbot comience a funcionar cuando ejecutes el código
chatbot()


Chatbot 🤖: ¡Hola! Bienvenido a Soft Art, Soy tu asistente virtual.
💻Si estás buscando contratar un servicio de software, estás en el lugar correcto.
Tú: salir
Chatbot 🤖: ¡Gracias por visitarnos! 👋
